In [1]:
import pandas as pd
import requests
import os
import time
from time import sleep
import random
import json
import hashlib

In [2]:
headers = {
    # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'Cookie': 'logan_session_token=8wa33wlkixrnw6fup8s6; logan_custom_report=; pvhistory="6L+U5ZuePjo8L3N1Z2dlc3QvZ2V0SnNvbkRhdGE/ZGV2aWNlX3N5c3RlbT1NQUNJTlRPU0gmeW9kYVJlYWR5PWg1JmNzZWNwbGF0Zm9ybT00JmNzZWN2ZXJzaW9uPTIuNC4wPjo8MTcxNTUxOTU0MDkwMV1fWw=="; m_flash2=1; _lxsdk_cuid=18f6ceece87c8-0cc2207c3e923e-1b525637-1fa400-18f6ceece87c8; _lxsdk=18f6ceece87c8-0cc2207c3e923e-1b525637-1fa400-18f6ceece87c8; _hc.v=e387f986-75d1-7b6f-c723-77863aa84234.1715519541; WEBDFPID=uuvy22679u9059wwy814x98v7uuw61u581u3xvvxu0y97958wu1uz800-2030879541033-1715519539854OMAAKCC75613c134b6a252faa6802015be905511940; GSI=#rf_syjgwlb@220829a#sl_zdjlwxbdsb@211011l#rf_syztwlb@210720a#rf_tsmbyrk@220309b',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9,zh;q=0.8,zh-CN;q=0.7',
    'Sec-Ch-Ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': "macOS",
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
}

In [3]:
base_url = 'https://m.dianping.com/myshow/ajax/performances/%s;st=0;p=%s;s=10;tft=0?cityId=%s&sellChannel=7'
detail_url = 'http://www.gewara.com/detail/'

In [4]:
meta_df = pd.DataFrame(columns=['url', 'pageNo', 'pageSize', 'totalHits', 'hasMore'])
details_df = pd.DataFrame(columns=['url', 'performanceId', 'ticketStatus', 'categoryId', 'shopId', 'name', 'shopName', 'lat', 'lng', 'address', 'posterUrl', 'postUrlForShare', 'showTimeRange', 'priceRange', 'sellPriceRange', 'seatUrl', 'limit', 'inventoryCount', 'tipForList', 'performanceLabelVO', 'detail', 'travelGuide', 'summary', 'ticketNotes', 'cityId', 'cityName', 'lowestPrice', 'lowestTicketPrice', 'lowestSellPrice', 'performanceTag', 'minDiscount', 'seatType', 'saleRemindVO', 'shareLink', 'stockOutRegister', 'needAnswer', 'questionBankId', 'questionHint', 'isCurrentCity', 'distance', 'poi', 'self', 'thirdPartyDesc', 'bsPerformanceId', 'tpId', 'needPreInput', 'kingHonorParam', 'needRemindPrice', 'ticketPriceList', 'currentLimit', 'modelStyle', 'only', 'needRealName', 'maxBuyLimitPerId', 'generalAgent', 'exclusive', 'maxBuyLimitPerOrder', 'maxBuyLimitPerUser', 'projectVideoVO', 'isAssemble', 'projectDetailAssembleVO', 'priceRangeType', 'projectExtendVO', 'preview', 'shareTitle', 'promotional', 'needCloseDetail', 'needCloseRecommend', 'brightPointList', 'photoStageList', 'videoList', 'musicList', 'serviceTitleList', 'buttonToast', 'brightName', 'videoStageName', 'videoStagePhotoTotal', 'videoNum', 'photoNum', 'projectServing', 'projectStartTime', 'projectEndTime', 'robType', 'celebrityVOS', 'memberInfo', 'intendedNum', 'reachRobStandard', 'unusualSaleStatus', 'relievedTagVO', 'tags', 'recommendStatus', 'minTicketOrSellPrice', 'multipleReward', 'showMemReward', 'score', 'star', 'shortName', 'normalTags', 'playTime', 'playerMinNumLimit', 'playerMaxNumLimit', 'saleShowNum', 'saleType', 'labelId', 'cornerDisplayName', 'topType', 'billBoardVO', 'buyInstructionType', 'discountTag', 'condRefund', 'brandId', 'brandName', 'couponDiscount', 'couponMinSellPrice', 'couponTicketPrice', 'couponTicketCount', 'couponActivityVO', 'couponActivityTagVO', 'couponItemVOList', 'isNewMode', 'jumpDetailUrl', 'kingHonorFlag', 'stockOut'])

In [5]:
category_list = [1, 10]
city_list = [1, 10, 20, 30, 40, 42, 44, 45, 50, 51, 52, 55, 56, 57, 59, 62, 66, 70, 73, 80]
# city_list = [59]

In [73]:
for category in category_list:
    for city in city_list:
        print('正在爬取 category: %s, city: %s' % (str(category), str(city)))
        url = base_url % (str(category),'1',str(city))
        r = requests.get(url, headers=headers)
        json_r = json.loads(r.text)

        new_meta_row = {'url': url}
        for col_ in meta_df.columns[1:]:
            new_meta_row[col_] = json_r['paging'][col_]
        meta_df = pd.concat([meta_df, pd.DataFrame([new_meta_row])], ignore_index=True)

        for detail_data in json_r['data']:
            new_detail_row = {'url': detail_url + str(detail_data['performanceId'])}
            for col_ in details_df.columns[1:]:
                if col_ == 'performanceLabelVO':
                    new_detail_row[col_] = str(detail_data[col_])
                else:
                    new_detail_row[col_] = detail_data[col_]
            details_df = pd.concat([details_df, pd.DataFrame([new_detail_row])], ignore_index=True)
            

        while json_r['paging']['hasMore'] == True:
            pageNo = json_r['paging']['pageNo'] + 1
            url = base_url % (str(category),str(pageNo),str(city))
            r = requests.get(url, headers=headers)
            json_r = json.loads(r.text)

            new_meta_row = {'url': url}
            for col_ in meta_df.columns[1:]:
                new_meta_row[col_] = json_r['paging'][col_]
            meta_df = pd.concat([meta_df, pd.DataFrame([new_meta_row])], ignore_index=True)

            for detail_data in json_r['data']:
                new_detail_row = {'url': detail_url + str(detail_data['performanceId'])}
                for col_ in details_df.columns[1:]:
                    if col_ == 'performanceLabelVO':
                        new_detail_row[col_] = str(detail_data[col_])
                    else:
                        new_detail_row[col_] = detail_data[col_]
                details_df = pd.concat([details_df, pd.DataFrame([new_detail_row])], ignore_index=True)
            sleep(random.randint(1, 3))
        sleep(random.randint(1, 3))


正在爬取 category: 1, city: 1


/tmp/ipykernel_1632834/601027356.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  details_df = pd.concat([details_df, pd.DataFrame([new_detail_row])], ignore_index=True)


正在爬取 category: 1, city: 10
正在爬取 category: 1, city: 20
正在爬取 category: 1, city: 30
正在爬取 category: 1, city: 40
正在爬取 category: 1, city: 42
正在爬取 category: 1, city: 44
正在爬取 category: 1, city: 45
正在爬取 category: 1, city: 50
正在爬取 category: 1, city: 51
正在爬取 category: 1, city: 52
正在爬取 category: 1, city: 55
正在爬取 category: 1, city: 56
正在爬取 category: 1, city: 57
正在爬取 category: 1, city: 59
正在爬取 category: 1, city: 62
正在爬取 category: 1, city: 66
正在爬取 category: 1, city: 70
正在爬取 category: 1, city: 73
正在爬取 category: 1, city: 80
正在爬取 category: 10, city: 1
正在爬取 category: 10, city: 10
正在爬取 category: 10, city: 20
正在爬取 category: 10, city: 30
正在爬取 category: 10, city: 40
正在爬取 category: 10, city: 42
正在爬取 category: 10, city: 44
正在爬取 category: 10, city: 45
正在爬取 category: 10, city: 50
正在爬取 category: 10, city: 51
正在爬取 category: 10, city: 52
正在爬取 category: 10, city: 55
正在爬取 category: 10, city: 56
正在爬取 category: 10, city: 57
正在爬取 category: 10, city: 59
正在爬取 category: 10, city: 62
正在爬取 category: 10, city: 66
正在爬取 categor

In [74]:
meta_df

,url,pageNo,pageSize,totalHits,hasMore
0,https://m.dianping.com/myshow/ajax/performance...,1,10,40,True
1,https://m.dianping.com/myshow/ajax/performance...,2,10,40,True
2,https://m.dianping.com/myshow/ajax/performance...,3,10,40,True
3,https://m.dianping.com/myshow/ajax/performance...,4,10,40,False
4,https://m.dianping.com/myshow/ajax/performance...,1,10,54,True
...,...,...,...,...,...
60,https://m.dianping.com/myshow/ajax/performance...,1,10,10,False
61,https://m.dianping.com/myshow/ajax/performance...,1,10,6,False
62,https://m.dianping.com/myshow/ajax/performance...,1,10,5,False
63,https://m.dianping.com/myshow/ajax/performance...,1,10,10,False


In [75]:
details_df

,url,performanceId,ticketStatus,categoryId,shopId,name,shopName,lat,lng,address,...,couponMinSellPrice,couponTicketPrice,couponTicketCount,couponActivityVO,couponActivityTagVO,couponItemVOList,isNewMode,jumpDetailUrl,kingHonorFlag,stockOut
0,http://www.gewara.com/detail/318851,318851,2,1,130078411,2024 郁可唯“终身浪漫的开始”演唱会-北京站,国家体育馆,39.996339,116.390734,天辰东路9号,...,None,None,None,None,None,None,0,/pages/show/detail/index?id=318851&isNewPage=f...,False,False
1,http://www.gewara.com/detail/322688,322688,2,1,98025245,重塑雕像的权利【喝彩之后】2024北京专场演唱会,华熙LIVE·五棵松,39.911388,116.281239,万寿路街道复兴路69号,...,None,None,None,None,None,None,0,/pages/show/detail/index?id=322688&isNewPage=f...,False,False
2,http://www.gewara.com/detail/322333,322333,2,1,130078411,黄绮珊「晚霞·不晚」2024演唱会-北京站,国家体育馆,39.996339,116.390734,天辰东路9号,...,None,None,None,None,None,None,0,/pages/show/detail/index?id=322333&isNewPage=f...,False,False
3,http://www.gewara.com/detail/317485,317485,3,1,1000355484,徐佳莹「变得有些奢侈的事」演唱会-北京站,首都体育馆,39.940414,116.327440,中关村南大街56号,...,None,None,None,None,None,None,0,/pages/show/detail/index?id=317485&isNewPage=f...,False,False
4,http://www.gewara.com/detail/285566,285566,3,1,1848768654,【沉浸式演出】音乐live现场｜乐队的每天【派对专场】流行金曲｜嗨歌联盟-巡演呈现,雍和宫·爱笑艺术剧院,39.948271,116.420326,安定门东大街28号雍和大厦西楼E座1层西南侧,...,None,None,None,None,None,None,0,/pages/show/detail/index?id=285566&isNewPage=f...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,http://www.gewara.com/detail/322859,322859,2,10,1031782560,爱玛追星·河流音乐节,祥源花世界-河流音乐节,31.687722,117.122289,金寨南路8号祥源花世界景区摩天轮户外大草坪,...,None,None,None,None,None,None,0,/pages/show/detail/index?id=322859&isNewPage=f...,False,False
574,http://www.gewara.com/detail/323506,323506,2,10,698730923,蚌埠涂山音乐嘉年华,蚌埠市体育中心体育场,32.892404,117.368893,燕山路与解放路交叉口东南100米,...,None,None,None,None,None,None,0,/pages/show/detail/index?id=323506&isNewPage=f...,False,False
575,http://www.gewara.com/detail/320240,320240,2,10,1129866004,2024采石矶长江音乐节,2024采石矶长江音乐节,31.650510,118.459300,唐贤街1号,...,None,None,None,None,None,None,0,/pages/show/detail/index?id=320240&isNewPage=f...,False,False
576,http://www.gewara.com/detail/324046,324046,3,10,1135690121,西湖音乐节,兔子山国际音乐公园,30.186987,120.053265,上城埭路,...,None,None,None,None,None,None,0,/pages/show/detail/index?id=324046&isNewPage=f...,False,True


In [76]:
# save to csv file
meta_df.to_csv('gewara/meta-20240512.csv', index=False)
details_df.to_csv('gewara/details-20240512.csv', index=False)

# 爬取具体Docs

In [4]:
details_df = pd.read_csv('gewara/details-20240512.csv')
# details_df

In [6]:
# details_df

In [5]:
details_df['categoryId'].value_counts()

# 1: 演唱会
# 2: 体育
# 3: 戏曲戏剧
# 4: 音乐剧
# 5: 话剧舞剧
# 6: 音乐会
# 7: 亲子互动
# 8: ？《宋美龄耳环丢了》张园纪念馆沉浸式夜游戏剧
# 9: 展览
# 10: 音乐节

categoryId
1     390
10    188
Name: count, dtype: int64

In [5]:
detail_id_list = list(details_df['performanceId'])
len(detail_id_list)

578

In [6]:
import os
exist_files_1 = os.listdir('./gewara/details')
exist_ids = [int(file.split('.')[0]) for file in exist_files_1]
print(len(exist_ids))

337


In [13]:
problem_id_list = []

In [51]:
problem_id_list

[]

In [54]:
len(problem_id_list)

339

In [32]:
type(detail_id_list[38])

int

In [7]:
my_headers = [
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 "
]

headers = {
    # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299',
    # 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'User-Agent': 'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    'Cookie': 'logan_session_token=8wa33wlkixrnw6fup8s6; logan_custom_report=; pvhistory="6L+U5ZuePjo8L3N1Z2dlc3QvZ2V0SnNvbkRhdGE/ZGV2aWNlX3N5c3RlbT1NQUNJTlRPU0gmeW9kYVJlYWR5PWg1JmNzZWNwbGF0Zm9ybT00JmNzZWN2ZXJzaW9uPTIuNC4wPjo8MTcxNTUxOTU0MDkwMV1fWw=="; m_flash2=1; _lxsdk_cuid=18f6ceece87c8-0cc2207c3e923e-1b525637-1fa400-18f6ceece87c8; _lxsdk=18f6ceece87c8-0cc2207c3e923e-1b525637-1fa400-18f6ceece87c8; _hc.v=e387f986-75d1-7b6f-c723-77863aa84234.1715519541; WEBDFPID=uuvy22679u9059wwy814x98v7uuw61u581u3xvvxu0y97958wu1uz800-2030879541033-1715519539854OMAAKCC75613c134b6a252faa6802015be905511940; GSI=#rf_syjgwlb@220829a#sl_zdjlwxbdsb@211011l#rf_syztwlb@210720a#rf_tsmbyrk@220309b',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9,zh;q=0.8,zh-CN;q=0.7',
    'Sec-Ch-Ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': "macOS",
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
}

In [8]:
# agents = [
#     "Mozilla/5.0 (Linux; U; Android 2.3.6; en-us; Nexus S Build/GRK39F) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#     "Avant Browser/1.2.789rel1 (http://www.avantbrowser.com)",
#     "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.5 (KHTML, like Gecko) Chrome/4.0.249.0 Safari/532.5",
#     "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.9 (KHTML, like Gecko) Chrome/5.0.310.0 Safari/532.9",
#     "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7",
#     "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14",
#     "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14",
#     "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20",
#     "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.27 (KHTML, like Gecko) Chrome/12.0.712.0 Safari/534.27",
#     "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.24 Safari/535.1",
#     "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/15.0.874.120 Safari/535.2",
#     "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.36 Safari/535.7",
#     "Mozilla/5.0 (Windows; U; Windows NT 6.0 x64; en-US; rv:1.9pre) Gecko/2008072421 Minefield/3.0.2pre",
#     "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.10) Gecko/2009042316 Firefox/3.0.10",
#     "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-GB; rv:1.9.0.11) Gecko/2009060215 Firefox/3.0.11 (.NET CLR 3.5.30729)",
#     "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6 GTB5",
#     "Mozilla/5.0 (Windows; U; Windows NT 5.1; tr; rv:1.9.2.8) Gecko/20100722 Firefox/3.6.8 ( .NET CLR 3.5.30729; .NET4.0E)",
#     "Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
#     "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
#     "Mozilla/5.0 (Windows NT 5.1; rv:5.0) Gecko/20100101 Firefox/5.0",
#     "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0a2) Gecko/20110622 Firefox/6.0a2",
#     "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:7.0.1) Gecko/20100101 Firefox/7.0.1",
#     "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0b4pre) Gecko/20100815 Minefield/4.0b4pre",
#     "Mozilla/4.0 (compatible; MSIE 5.5; Windows NT 5.0 )",
#     "Mozilla/4.0 (compatible; MSIE 5.5; Windows 98; Win 9x 4.90)",
#     "Mozilla/5.0 (Windows; U; Windows XP) Gecko MultiZilla/1.6.1.0a",
#     "Mozilla/2.02E (Win95; U)",
#     "Mozilla/3.01Gold (Win95; I)",
#     "Mozilla/4.8 [en] (Windows NT 5.1; U)",
#     "Mozilla/5.0 (Windows; U; Win98; en-US; rv:1.4) Gecko Netscape/7.1 (ax)",
#     "HTC_Dream Mozilla/5.0 (Linux; U; Android 1.5; en-ca; Build/CUPCAKE) AppleWebKit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari/525.20.1",
#     "Mozilla/5.0 (hp-tablet; Linux; hpwOS/3.0.2; U; de-DE) AppleWebKit/534.6 (KHTML, like Gecko) wOSBrowser/234.40.1 Safari/534.6 TouchPad/1.0",
#     "Mozilla/5.0 (Linux; U; Android 1.5; en-us; sdk Build/CUPCAKE) AppleWebkit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari/525.20.1",
#     "Mozilla/5.0 (Linux; U; Android 2.1; en-us; Nexus One Build/ERD62) AppleWebKit/530.17 (KHTML, like Gecko) Version/4.0 Mobile Safari/530.17",
#     "Mozilla/5.0 (Linux; U; Android 2.2; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#     "Mozilla/5.0 (Linux; U; Android 1.5; en-us; htc_bahamas Build/CRB17) AppleWebKit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari/525.20.1",
#     "Mozilla/5.0 (Linux; U; Android 2.1-update1; de-de; HTC Desire 1.19.161.5 Build/ERE27) AppleWebKit/530.17 (KHTML, like Gecko) Version/4.0 Mobile Safari/530.17",
#     "Mozilla/5.0 (Linux; U; Android 2.2; en-us; Sprint APA9292KT Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#     "Mozilla/5.0 (Linux; U; Android 1.5; de-ch; HTC Hero Build/CUPCAKE) AppleWebKit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari/525.20.1",
#     "Mozilla/5.0 (Linux; U; Android 2.2; en-us; ADR6300 Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#     "Mozilla/5.0 (Linux; U; Android 2.1; en-us; HTC Legend Build/cupcake) AppleWebKit/530.17 (KHTML, like Gecko) Version/4.0 Mobile Safari/530.17",
#     "Mozilla/5.0 (Linux; U; Android 1.5; de-de; HTC Magic Build/PLAT-RC33) AppleWebKit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari/525.20.1 FirePHP/0.3",
#     "Mozilla/5.0 (Linux; U; Android 1.6; en-us; HTC_TATTOO_A3288 Build/DRC79) AppleWebKit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari/525.20.1",
#     "Mozilla/5.0 (Linux; U; Android 1.0; en-us; dream) AppleWebKit/525.10  (KHTML, like Gecko) Version/3.0.4 Mobile Safari/523.12.2",
#     "Mozilla/5.0 (Linux; U; Android 1.5; en-us; T-Mobile G1 Build/CRB43) AppleWebKit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari 525.20.1",
#     "Mozilla/5.0 (Linux; U; Android 1.5; en-gb; T-Mobile_G2_Touch Build/CUPCAKE) AppleWebKit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari/525.20.1",
#     "Mozilla/5.0 (Linux; U; Android 2.0; en-us; Droid Build/ESD20) AppleWebKit/530.17 (KHTML, like Gecko) Version/4.0 Mobile Safari/530.17",
#     "Mozilla/5.0 (Linux; U; Android 2.2; en-us; Droid Build/FRG22D) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#     "Mozilla/5.0 (Linux; U; Android 2.0; en-us; Milestone Build/ SHOLS_U2_01.03.1) AppleWebKit/530.17 (KHTML, like Gecko) Version/4.0 Mobile Safari/530.17",
#     "Mozilla/5.0 (Linux; U; Android 2.0.1; de-de; Milestone Build/SHOLS_U2_01.14.0) AppleWebKit/530.17 (KHTML, like Gecko) Version/4.0 Mobile Safari/530.17",
#     "Mozilla/5.0 (Linux; U; Android 3.0; en-us; Xoom Build/HRI39) AppleWebKit/525.10  (KHTML, like Gecko) Version/3.0.4 Mobile Safari/523.12.2",
#     "Mozilla/5.0 (Linux; U; Android 0.5; en-us) AppleWebKit/522  (KHTML, like Gecko) Safari/419.3",
#     "Mozilla/5.0 (Linux; U; Android 1.1; en-gb; dream) AppleWebKit/525.10  (KHTML, like Gecko) Version/3.0.4 Mobile Safari/523.12.2",
#     "Mozilla/5.0 (Linux; U; Android 2.0; en-us; Droid Build/ESD20) AppleWebKit/530.17 (KHTML, like Gecko) Version/4.0 Mobile Safari/530.17",
#     "Mozilla/5.0 (Linux; U; Android 2.1; en-us; Nexus One Build/ERD62) AppleWebKit/530.17 (KHTML, like Gecko) Version/4.0 Mobile Safari/530.17",
#     "Mozilla/5.0 (Linux; U; Android 2.2; en-us; Sprint APA9292KT Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#     "Mozilla/5.0 (Linux; U; Android 2.2; en-us; ADR6300 Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#     "Mozilla/5.0 (Linux; U; Android 2.2; en-ca; GT-P1000M Build/FROYO) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#     "Mozilla/5.0 (Linux; U; Android 3.0.1; fr-fr; A500 Build/HRI66) AppleWebKit/534.13 (KHTML, like Gecko) Version/4.0 Safari/534.13",
#     "Mozilla/5.0 (Linux; U; Android 3.0; en-us; Xoom Build/HRI39) AppleWebKit/525.10  (KHTML, like Gecko) Version/3.0.4 Mobile Safari/523.12.2",
#     "Mozilla/5.0 (Linux; U; Android 1.6; es-es; SonyEricssonX10i Build/R1FA016) AppleWebKit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari/525.20.1",
#     "Mozilla/5.0 (Linux; U; Android 1.6; en-us; SonyEricssonX10i Build/R1AA056) AppleWebKit/528.5  (KHTML, like Gecko) Version/3.1.2 Mobile Safari/525.20.1",
# ]


agents_2 = [
        "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
        "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
        "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)",
        "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
        "Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
        "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11",
        "Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Avant Browser)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
        "Mozilla/5.0 (iPhone; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
        "Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
        "Mozilla/5.0 (iPad; U; CPU OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
        "Mozilla/5.0 (Linux; U; Android 2.3.7; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
        "MQQBrowser/26 Mozilla/5.0 (Linux; U; Android 2.3.7; zh-cn; MB200 Build/GRJ22; CyanogenMod-7) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
        "Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10",
        "Mozilla/5.0 (Linux; U; Android 3.0; en-us; Xoom Build/HRI39) AppleWebKit/534.13 (KHTML, like Gecko) Version/4.0 Safari/534.13",
        "Mozilla/5.0 (BlackBerry; U; BlackBerry 9800; en) AppleWebKit/534.1+ (KHTML, like Gecko) Version/6.0.0.337 Mobile Safari/534.1+",
        "Mozilla/5.0 (hp-tablet; Linux; hpwOS/3.0.0; U; en-US) AppleWebKit/534.6 (KHTML, like Gecko) wOSBrowser/233.70 Safari/534.6 TouchPad/1.0",
        "Mozilla/5.0 (SymbianOS/9.4; Series60/5.0 NokiaN97-1/20.0.019; Profile/MIDP-2.1 Configuration/CLDC-1.1) AppleWebKit/525 (KHTML, like Gecko) BrowserNG/7.1.18124",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0; HTC; Titan)",
        "UCWEB7.0.2.37/28/999",
        "NOKIA5700/ UCWEB7.0.2.37/28/999",
        "Openwave/ UCWEB7.0.2.37/28/999",
        "Mozilla/4.0 (compatible; MSIE 6.0; ) Opera/UCWEB7.0.2.37/28/999",
        # iPhone 6：
        "Mozilla/6.0 (iPhone; CPU iPhone OS 8_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/8.0 Mobile/10A5376e Safari/8536.25",
]

In [37]:
# for i, exist_id in enumerate(exist_ids):
#     json_docs = {}
#     for col_ in details_df.columns:
#         json_docs[col_] = str(details_df[details_df['performanceId']==exist_id][col_].values[0])
#     # print("json_docs", json_docs)
#     json_before = {}
#     with open('./gewara/details/%s.json' % str(exist_id), 'r') as f:
#         json_before = json.load(f)
#     # print("json_before", json_before)
#     json_docs.update(json_before)

#     with open('./gewara/details/%s.json' % str(exist_id), 'w') as f:
#         json.dump(json_docs, f, indent=4, ensure_ascii=False)

In [9]:
# import urllib2 as ulb

# proxy_list = [
#     '183.95.80.102:8080',
#     '123.160.31.71:8080',
#     '115.231.128.79:8080',
#     '166.111.77.32:80',
#     '43.240.138.31:8080',
#     '218.201.98.196:3128'
# ]

proxy = 'http://127.0.0.1:6555'

In [10]:
from bs4 import BeautifulSoup

for i, detail_id in enumerate(detail_id_list):
    if detail_id in exist_ids:
        continue
    print(f'正在爬取第{i}个 {detail_id}')
    detail_id_url = detail_url + str(detail_id)

    headers['User-Agent'] = random.choice(agents_2)
    r = requests.get(detail_id_url, headers=headers, proxies={'http': proxy})

    soup = BeautifulSoup(r.text, 'html.parser')

    content_soup = soup.find_all('section', id='mainContent')
    
    fail_cnt = 0
    if len(content_soup) == 0:
        for h in agents_2:
            headers['User-Agent'] = h
            r = requests.get(detail_id_url, headers=headers, proxies={'http': proxy})
            soup = BeautifulSoup(r.text, 'html.parser')
            content_soup = soup.find_all('section', id='mainContent')
            if len(content_soup) == 0:
                fail_cnt += 1
        if fail_cnt > 20:
            print(f'no data in {i} - {detail_id}, fail_cnt: {fail_cnt}')
            # problem_id_list.append(detail_id)
            # break
            problem_id_list.append(detail_id)

            print(f'problem_id_list append {i} - {detail_id}')
            break
        # 
        continue
        
    
    content_soup = content_soup[0]

    img_list = []
    img_soup = content_soup.find_all('img')
    for img in img_soup:
        img_list.append(img.get('src'))

    text_list = []
    span_soup = content_soup.find_all('span') 
    for span in span_soup:
        text_list.append(span.text)

    json_docs = {}
    for col_ in details_df.columns:
        json_docs[col_] = str(details_df[details_df['performanceId']==detail_id][col_].values[0])
    json_details = { 
        'img_list': img_list,
        'text_list': text_list
    }
    json_docs.update(json_details)

    with open(f'gewara/details/{detail_id}.json', 'w') as f:
        json.dump(json_docs, f, indent=4, ensure_ascii=False)
    print(f'第{i}个 {detail_id} 爬取完成')
    sleep(random.randint(1, 2))

正在爬取第348个 309269


KeyboardInterrupt: 

In [11]:
headers = {
    # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299',
    # 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'User-Agent': random.choice(my_headers),
    'Cookie': 'logan_session_token=8wa33wlkixrnw6fup8s6; logan_custom_report=; pvhistory="6L+U5ZuePjo8L3N1Z2dlc3QvZ2V0SnNvbkRhdGE/ZGV2aWNlX3N5c3RlbT1NQUNJTlRPU0gmeW9kYVJlYWR5PWg1JmNzZWNwbGF0Zm9ybT00JmNzZWN2ZXJzaW9uPTIuNC4wPjo8MTcxNTUxOTU0MDkwMV1fWw=="; m_flash2=1; _lxsdk_cuid=18f6ceece87c8-0cc2207c3e923e-1b525637-1fa400-18f6ceece87c8; _lxsdk=18f6ceece87c8-0cc2207c3e923e-1b525637-1fa400-18f6ceece87c8; _hc.v=e387f986-75d1-7b6f-c723-77863aa84234.1715519541; WEBDFPID=uuvy22679u9059wwy814x98v7uuw61u581u3xvvxu0y97958wu1uz800-2030879541033-1715519539854OMAAKCC75613c134b6a252faa6802015be905511940; GSI=#rf_syjgwlb@220829a#sl_zdjlwxbdsb@211011l#rf_syztwlb@210720a#rf_tsmbyrk@220309b',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9,zh;q=0.8,zh-CN;q=0.7',
    'Sec-Ch-Ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': "macOS",
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
}

test_url = 'http://www.gewara.com/detail/309269'
test_r = requests.get(test_url, headers=headers, proxies={'http': proxy})
# r = requests.get(detail_id_url, headers=headers, proxies={'http': proxy})

test_r.text

KeyboardInterrupt: 

In [93]:
from bs4 import BeautifulSoup


soup = BeautifulSoup(test_r.text, 'html.parser')
soup = soup.find_all('section', id='mainContent')[0]

img_list = []
img_soup = soup.find_all('img')
for img in img_soup:
    img_list.append(img.get('src'))

text_list = []
span_soup = soup.find_all('span') 
for span in span_soup:
    text_list.append(span.text)

doc_json = { 
    'img_list': img_list,
    'text_list': text_list
}

with open('gewara/test.json', 'w') as f:
    json.dump(doc_json, f, indent=4, ensure_ascii=False)

In [87]:
strong_soup = soup.find_all('strong') 
strong_soup

[<strong class="jsx-3462127015 jsx-3462127015">购票</strong>,
 <strong style="color: rgb(0, 0, 0);">郁可唯《终身浪漫的开始》2024 年巡回演唱会继续唱响！</strong>,
 <strong style="color: rgb(0, 0, 0);">《终身浪漫的开始》巡回演唱会——继续唱响！</strong>]

'订单动态请在微信小程序端查看'

In [62]:
details_df['projectDetailAssembleVO'].unique()
# 'distance', 'poi', 'self', 'thirdPartyDesc', 'bsPerformanceId', 'tpId', 'needPreInput', 'kingHonorParam', 'needRemindPrice', 'ticketPriceList', 'currentLimit', 'modelStyle', 'only', 'needRealName', 'maxBuyLimitPerId', 'generalAgent', 'exclusive', 'maxBuyLimitPerOrder', 'maxBuyLimitPerUser', 'projectVideoVO', 'isAssemble', 'projectDetailAssembleVO', 'priceRangeType', 'projectExtendVO', 'preview', 'shareTitle', 'promotional', 'needCloseDetail', 'needCloseRecommend', 'brightPointList', 'photoStageList', 'videoList', 'musicList', 'serviceTitleList', 'buttonToast', 'brightName', 'videoStageName', 'videoStagePhotoTotal', 'videoNum', 'photoNum', 'projectServing', 'projectStartTime', 'projectEndTime', 'robType', 'celebrityVOS', 'memberInfo', 'intendedNum', 'reachRobStandard', 'unusualSaleStatus', 'relievedTagVO', 'tags', 'recommendStatus', 'minTicketOrSellPrice', 'multipleReward', 'showMemReward', 'score', 'star', 'shortName', 'normalTags', 'playTime', 'playerMinNumLimit', 'playerMaxNumLimit', 'saleShowNum', 'saleType', 'labelId', 'cornerDisplayName', 'topType', 'billBoardVO', 'buyInstructionType', 'discountTag'

array([None], dtype=object)